<a href="https://colab.research.google.com/github/DonguYang/Data-Analysis/blob/circleci-project-setup/%08KPI%EB%B3%84_%EB%B3%B4%EA%B3%A0%EC%84%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 노출수, 클릭수, 소진금액, 전환수, 평균구매액, 세션매출, 직접매출, 총매출

In [ ]:
import pandas as pd
from datetime import date, timedelta, datetime
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pymysql
from sqlalchemy import create_engine, text
import time


In [ ]:
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']  # 필요한 스코프 추가
creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/enliple/Desktop/python1/gspread-407309-1bab071f32e2.json', scope)
client = gspread.authorize(creds)


In [ ]:
code = [
    35849, 35848, 35847, 35846, 35845, 35844, 35843, 35842, 35841, 35840,
    35839, 35838, 35855, 35852, 35854, 35862, 35864, 35863, 35861, 35860,
    35859, 35858, 35857, 35856, 35853, 35851, 35850, 36381, 36380, 36379,
    36365, 36378, 36377, 36376, 36364, 36375, 36374, 36373, 36363, 36386,
    36388, 36383, 36387, 36394, 36396, 36392, 36395, 36400
]

code_str = ', '.join(str(num) for num in code)


In [ ]:
# dreamsearch를 사용할 시, db 부분을 바꾼 코드 별도의 코드 추가

host = "192.168.2.82"
user = "duyang"
password = "duyang2023!"
db = "BILLING"
charset = "utf8"

db_uri = f"mysql+pymysql://{user}:{password}@{host}/{db}?charset={charset}"
engine = create_engine(db_uri)

In [ ]:
today = datetime.today() - timedelta(1)

# 3일 전부터 오늘까지의 날짜를 갖는 리스트 생성
date_list = [(today - timedelta(days=i)).strftime("%Y%m%d") for i in range(2, -1, -1)]

date_list

['20231216', '20231217', '20231218']

In [ ]:
# DB에서 데이터 받아오기(오늘 기준 4일치 데이터를 매번 갱신)

# 오늘의 데이터를 받아오기 위한 것

query1 = f"""
    SELECT
        KPI_NO,
        STATS_DTTM,
        SUM(TOT_EPRS_CNT) AS TOT_EPRS_CNT,
        SUM(CLICK_CNT) AS CLICK_CNT,
        SUM(ADVRTS_AMT) AS ADVRTS_AMT
    FROM MOB_KPI_STATS
    WHERE STATS_DTTM IN ({', '.join(date_list)})
        AND KPI_NO IN ({code_str})
    GROUP BY KPI_NO, STATS_DTTM
"""

df1= pd.read_sql(query1, con=engine)

# 매출관련 불러오기



query2 = f"""
    SELECT KPI_NO, STATS_DTTM
                    , SUM(ORDER_CNT) AS ORDER_CNT
                    , SUM(ORDER_AMT) AS TOTAL_SALES
                    , SUM(IF(SESION_SELNG_YN = 'Y', ORDER_AMT,0)) AS SESSION_SALES
                    , SUM(IF(DIRECT_YN = 'Y', ORDER_AMT,0)) AS DIRECT_SALES
      FROM MOB_CNVRS_RENEW_KPI_STATS
     WHERE STATS_DTTM IN ({', '.join(date_list)})
       AND KPI_NO IN ({code_str})
GROUP BY STATS_DTTM, KPI_NO

"""
df2 = pd.read_sql(query2, con=engine)

table1 = pd.merge(df1, df2, on=['KPI_NO', 'STATS_DTTM'], how='outer')

table1 = table1.fillna(0)

table1.shape

(135, 9)

In [ ]:
table1.STATS_DTTM.value_counts()

STATS_DTTM
20231216    45
20231217    45
20231218    45
Name: count, dtype: int64

In [ ]:
for i in date_list:
    print(table1[table1['STATS_DTTM'] == int(i)].shape[0])

45
45
45


In [ ]:
code_str = [str(i) for i in code]

In [ ]:
# table 정렬
code_int = [int(code_val) for code_val in code_str]
missing_kpi_no = [kpi_no for kpi_no in code_int if kpi_no not in table1['KPI_NO'].values]
missing_kpi_no_count = len(missing_kpi_no)

if missing_kpi_no:
    new_rows = []
    for kpi_no in missing_kpi_no:
        new_row = {'KPI_NO': kpi_no, 'STATS_DTTM': 0, 'TOT_EPRS_CNT': 0, 'CLICK_CNT': 0, 'ADVRTS_AMT': 0, 'ORDER_CNT': 0, 'TOTAL_SALES': 0, 'SESSION_SALES': 0, 'DIRECT_SALES': 0}
        new_rows.append(new_row)

    table1 = pd.concat([table1, pd.DataFrame(new_rows)], ignore_index=True)

table1 = table1.set_index('KPI_NO').loc[code_int].reset_index()

missing_stats_dttm = table1[table1['STATS_DTTM'] == 0]

# 3개의 누락된 행을 9개로 확장
missing_stats_dttm_expanded = pd.concat([missing_stats_dttm] * 3).reset_index(drop=True)

# date_list를 3개씩 3번 반복하여 9개의 날짜를 생성 (missing value의 갯수에 맞춰 곱해주기)
date_list_ex = list(map(int, date_list * missing_kpi_no_count))

# 확장된 행에 날짜 데이터 추가
missing_stats_dttm_expanded['STATS_DTTM'] = date_list_ex

# 기존 행 제거하고 확장된 행 추가
table1 = table1[table1['STATS_DTTM'] != 0]
table1 = pd.concat([table1, missing_stats_dttm_expanded], ignore_index=True)

# 소진금액 소수점 처리
table1['ADVRTS_AMT'] = table1['ADVRTS_AMT'].astype(int)

table1 = table1.set_index('KPI_NO').loc[code_int].reset_index()

table1.shape

(144, 9)

In [ ]:
for i in date_list:
    print(table1[table1['STATS_DTTM'] == int(i)].shape[0])

48
48
48


In [ ]:
table1[table1['STATS_DTTM'] == 20231209].to_excel('table1.xlsx')

In [ ]:
sheet_list = ['노출수raw', '클릭수raw','소진금액raw','전환수raw','세션매출raw','직접매출raw','총매출raw']
val_list = ['TOT_EPRS_CNT','CLICK_CNT','ADVRTS_AMT','ORDER_CNT','SESSION_SALES','DIRECT_SALES','TOTAL_SALES']
data_dict = {sheet_list[i]: val_list[i] for i in range(len(sheet_list))}

def write_data_to_sheet(sheet_name, table_data, data_dict, today):
    # 클라이언트에서 스프레드시트 가져오기
    sheet = client.open('24Q1_어도비_모비온 성과공유').worksheet(sheet_name)

    # 3일 전 날짜 계산
    start_date = today
    base_date = datetime.strptime("20231205", "%Y%m%d")

    # 선택된 열 범위 설정
    start_row = 5
    end_row = 52

    # 데이터를 스프레드시트에 쓰기
    for i in range(3):  # 3일 전부터 오늘까지의 데이터를 처리
        selected_date = (start_date + timedelta(days=i-2)).strftime('%Y%m%d')  # 3일 전부터 시작하여 하루씩 증가
        selected_column = 2 + i +(start_date - base_date).days

        cell_list = sheet.range(f'{chr(65 + selected_column)}{start_row}:{chr(65 + selected_column)}{end_row}')
        data_column = data_dict.get(sheet_name)  # 시트 이름을 기반으로 데이터 딕셔너리에서 해당하는 컬럼 선택
        data_to_write = table_data[table_data['STATS_DTTM'] == int(selected_date)][data_column]

        for j, value in enumerate(data_to_write):
            cell_list[j].value = value

        sheet.update_cells(cell_list)

In [ ]:
for sheet_name in sheet_list:
        write_data_to_sheet(sheet_name, table1, data_dict, today)